# Time Series Anomaly Detection
### *Codeup's Curriculum Access Logs*

#### Programs:
- Data Science
- Web Development

#### Goals for each program:
>1. Find lessons where the *most amount of traffic occurs*
2. Is there a cohort that referred to a lesson more than any other?
3. Are there any students, while active, who didn't access the curriculum much? 
    - If so, what can be said about these students?
4. Is there any suspicious activity, such as entities accessing the curriculum who aren't authorized? 
    - Does it appear that any web-scraping is happening? 
    - Are there any suspicious IP addresses?
    - Any odd user-agents?
5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. 
    - Do you see any evidence of that happening? 
    - Did it happen before? 
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
    - Which lessons are least accessed?
7. Anything else anomalous? 

---

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

from wrangle import wrangle_logs
import requests

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

---
## Wrangling

In [2]:
df, no_id = wrangle_logs()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 637296 entries, 2018-01-26 09:56:02 to 2020-11-02 16:48:47
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   page        637296 non-null  object        
 1   user_id     637296 non-null  int64         
 2   cohort_id   637296 non-null  int64         
 3   ip          637296 non-null  object        
 4   name        637296 non-null  object        
 5   start_date  637296 non-null  datetime64[ns]
 6   end_date    637296 non-null  datetime64[ns]
 7   program_id  637296 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 43.8+ MB


In [4]:
no_id.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44840 entries, 2018-01-26 16:46:16 to 2020-11-02 16:30:49
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   page       44840 non-null  object 
 1   user_id    44840 non-null  int64  
 2   cohort_id  44840 non-null  float64
 3   ip         44840 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.7+ MB


In [5]:
print('Curriculum Access History')
print(df.index.min()) 
print(df.index.max(), '\n')

print('Cohorts with no ID')
print(no_id.index.min()) 
print(no_id.index.max())

Curriculum Access History
2018-01-26 09:56:02
2020-11-02 16:48:47 

Cohorts with no ID
2018-01-26 16:46:16
2020-11-02 16:30:49


#### Web Development and Data Science

In [30]:
# created different data frames that split up each program
ds = df[df.program_id == 3]
wd = df[df.program_id != 3]

>**Summary**:
- 44840 null values in `cohort_id`
    - separated into "no_id" dataframe
- 674618 total entries with `cohort_id`
    - prepared with extra columns on
        - cohort `name`
        - `start_date`
        - `end_date`
        - `program_id`

---
## Exploration

<div class="alert alert-block alert-info">1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?</div>

### Web Development

In [8]:
pd.DataFrame(wd.page.value_counts().head())

,page
javascript-i,13867
toc,13350
search/search_index.json,12244
java-iii,10584
html-css,10320


### Data Science

In [9]:
pd.DataFrame(ds.page.value_counts().head())

,page
1-fundamentals/modern-data-scientist.jpg,1560
1-fundamentals/AI-ML-DL-timeline.jpg,1554
1-fundamentals/1.1-intro-to-data-science,1532
search/search_index.json,1330
6-regression/1-overview,1122


---
<div class="alert alert-block alert-info">2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over?</div>

### Web Development

In [10]:
pd.DataFrame(wd.groupby('name').page.value_counts().sort_values(ascending=False).head(10))

page
name     page                          
Zion     toc                       1455
Ceres    search/search_index.json  1372
Apex     search/search_index.json  1358
Fortuna  toc                       1261
Staff    javascript-i              1259
         search/search_index.json  1092
Ganymede search/search_index.json  1049
Staff    appendix                   986
Fortuna  search/search_index.json   985
Wrangell toc                        982

In [11]:
pd.DataFrame(wd[wd.page == 'toc'].groupby('name').page.value_counts().sort_values().head(10))

,,page
name,page,
Mammoth,toc,1
Badlands,toc,2
Joshua,toc,3
Hampton,toc,5
Niagara,toc,6
Ike,toc,6
Lassen,toc,10
Pinnacles,toc,11
Quincy,toc,12


In [12]:
pd.DataFrame(wd[wd.page == 'search/search_index.json'].groupby('name').page.value_counts().sort_values().head(10))

,,page
name,page,
Mammoth,search/search_index.json,1
Ike,search/search_index.json,1
Glacier,search/search_index.json,4
Kings,search/search_index.json,6
Quincy,search/search_index.json,6
Pinnacles,search/search_index.json,6
Niagara,search/search_index.json,7
Hampton,search/search_index.json,9
Olympic,search/search_index.json,17


### Data Science

In [13]:
pd.DataFrame(ds.groupby('name').page.value_counts().sort_values(ascending=False).head(10))

page
name   page                                          
Darden classification/overview                    756
Bayes  1-fundamentals/modern-data-scientist.jpg   625
       1-fundamentals/AI-ML-DL-timeline.jpg       623
       1-fundamentals/1.1-intro-to-data-science   614
Curie  6-regression/1-overview                    594
Darden classification/scale_features_or_not.svg   589
Bayes  search/search_index.json                   550
       6-regression/1-overview                    521
Darden sql/mysql-overview                         513
Curie  search/search_index.json                   480

In [14]:
pd.DataFrame(ds[ds.page == 'classification/overview'].groupby('name').page.value_counts().sort_values().head(10))

,,page
name,page,
Bayes,classification/overview,10
Curie,classification/overview,90
Darden,classification/overview,756


In [15]:
pd.DataFrame(ds[ds.page == '1-fundamentals/modern-data-scientist.jpg'].groupby('name').page.value_counts().sort_values().head(10))

,,page
name,page,
Curie,1-fundamentals/modern-data-scientist.jpg,467
Darden,1-fundamentals/modern-data-scientist.jpg,468
Bayes,1-fundamentals/modern-data-scientist.jpg,625


In [16]:
pd.DataFrame(ds[ds.page == '6-regression/1-overview'].groupby('name').page.value_counts().sort_values().head(10))

,,page
name,page,
Darden,6-regression/1-overview,7
Bayes,6-regression/1-overview,521
Curie,6-regression/1-overview,594


---
<div class="alert alert-block alert-info">3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?</div>

In [17]:
# filters df for observations only during the time the student 
# was 'active' for each program
active_access_wd = wd.loc[(wd.index >= wd.start_date) & (wd.index <= wd.end_date)]
active_access_ds = ds.loc[(ds.index >= ds.start_date) & (ds.index <= ds.end_date)]

### Web Development

In [18]:
# groups active web dev users by id and aggregates by page count and sorts them
low_access_wd = active_access_wd.groupby('user_id').size().sort_values().head()
low_access_wd

user_id
278     3
539     4
388     7
572    11
64     14
dtype: int64

In [19]:
# accessed pages of active web dev users from the list above
pd.DataFrame(active_access_wd[active_access_wd.user_id.isin(low_access_wd.index.tolist())])#[['user_id','end_date']])

,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-09-27 14:47:37,java-ii/arrays,278,24,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2
2018-09-27 14:58:48,java-ii/arrays,278,24,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2
2018-09-27 14:59:07,java-ii/collections,278,24,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,2
2019-03-19 09:50:23,html-css,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2
2019-03-19 09:50:28,html-css/elements,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2
2019-03-19 10:04:11,html-css/elements,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2
2019-03-19 10:19:32,html-css/elements,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2
2019-03-19 11:11:51,html-css/forms,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2
2019-03-19 11:12:02,html-css/elements,388,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2


In [20]:
# ip: 97.105.19.58
url = 'http://ip-api.com/csv/97.105.19.58'
response = requests.get(url)
location = response.text
location

'success,United States,US,TX,Texas,Dallas,75243,32.9103,-96.7283,America/Chicago,Spectrum,Codeup LLC,AS11427 Charter Communications Inc,97.105.19.58\n'

In [21]:
# ip: 107.77.217.9
url = 'http://ip-api.com/csv/107.77.217.9'
response = requests.get(url)
location = response.text
location

'success,United States,US,TX,Texas,Houston,77036,29.6968,-95.5446,America/Chicago,AT&T Mobility LLC,AT&T Mobility LLC,AS20057 AT&T Mobility LLC,107.77.217.9\n'

### Data Science

In [22]:
# groups active data sci users by id and aggregates by page count and sorts them
low_access_ds = active_access_ds.groupby('user_id').size().sort_values().head()
low_access_ds

user_id
679    10
697    12
487    16
785    29
780    44
dtype: int64

In [23]:
# accessed pages of active web dev users from the list above
active_access_ds[active_access_ds.user_id.isin(low_access_ds.index.tolist())]

,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2019-08-30 12:02:11,appendix/cli/1-intro,487,34,97.105.19.58,Bayes,2019-08-19,2020-01-30,3
2019-08-30 12:11:10,appendix/cli/2-listing-files,487,34,97.105.19.58,Bayes,2019-08-19,2020-01-30,3
2019-08-30 13:37:40,appendix/cli/0-overview,487,34,97.105.19.58,Bayes,2019-08-19,2020-01-30,3
2019-08-30 13:37:58,appendix/cli-git-overview,487,34,97.105.19.58,Bayes,2019-08-19,2020-01-30,3
2019-08-30 14:12:22,appendix/cli/3-file-paths,487,34,97.105.19.58,Bayes,2019-08-19,2020-01-30,3
...,...,...,...,...,...,...,...,...
2020-11-02 09:21:45,anomaly-detection/overview,780,59,99.132.128.255,Darden,2020-07-13,2021-01-12,3
2020-11-02 09:21:45,anomaly-detection/AnomalyDetectionCartoon.jpeg,780,59,99.132.128.255,Darden,2020-07-13,2021-01-12,3
2020-11-02 09:21:50,timeseries/modeling-lesson1,780,59,99.132.128.255,Darden,2020-07-13,2021-01-12,3


In [24]:
# ip's of active data science users' accessed pages
active_access_ds[active_access_ds.user_id.isin(low_access_ds.index.tolist())].ip.value_counts()

99.132.128.255    44
72.181.127.233    28
97.105.19.58      16
136.50.70.27      12
24.28.146.155     10
24.173.208.242     1
Name: ip, dtype: int64

In [25]:
# ip: 99.132.128.255
url = 'http://ip-api.com/csv/99.132.128.255'
response = requests.get(url)
location = response.text
location

'success,United States,US,TX,Texas,San Antonio,78202,29.4247,-98.4621,America/Chicago,"AT&T Services, Inc.",AT&T Corp,"AS7018 AT&T Services, Inc.",99.132.128.255\n'

>**Summary**
- For web dev:
    - there is an identical ip address for multiple user_ids and multiple cohorts. This just tells me that this is Codeup's location.
    - There was also another ip address based on a user from Houston with one occurence with Codeup's ip address (see first two rows in web dev dataframe)
- For data sci:
    - the ip addresses are more spread out. This make sense considering the work from home environment since the pandemic started

---
<div class="alert alert-block alert-info">4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents?</div>

In [26]:
no_id.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44840 entries, 2018-01-26 16:46:16 to 2020-11-02 16:30:49
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   page       44840 non-null  object 
 1   user_id    44840 non-null  int64  
 2   cohort_id  44840 non-null  float64
 3   ip         44840 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.7+ MB


In [27]:
# users with no cohort_id
no_id_users = no_id.groupby('user_id').size().index.to_list()
no_id_users

[48,
 54,
 58,
 59,
 61,
 62,
 63,
 64,
 73,
 74,
 78,
 79,
 86,
 88,
 89,
 100,
 103,
 111,
 137,
 166,
 176,
 213,
 247,
 317,
 346,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 372,
 375,
 403,
 406,
 429,
 544,
 644,
 663,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 731,
 736,
 744,
 782]

In [28]:
# users in original df that coincide with user_ids from the unknown cohort df
weird_users = df[df.user_id.isin(no_id_users)].user_id.value_counts().index.to_list()
weird_users

[64, 644, 375, 346, 358, 88, 663]

In [29]:
# users from above respective cohorts
pd.DataFrame(df[df.user_id.isin(weird_users)].groupby(['user_id', 'name']).size())

0
user_id name           
64      Arches     3538
        Europa       14
        Staff      9920
88      Glacier     326
        Ike           5
        Joshua        9
346     Sequoia      49
        Zion       1470
358     Bayes      1051
375     Andromeda  1553
644     Ganymede   1622
663     Hyperion    270

>**Summary**
- I checked out the users that didn't have cohort_ids to see if they had a similar user_id to any observations with cohort information. 
- I found 7 users with absent cohort_ids that shared a user_id that had a cohort_id.
- Finally, I discovered that 3 users_id's had information from 2 or three different cohorts (dataframe above).
- All findings are from web dev programs

---
<div class="alert alert-block alert-info">5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?</div>

---
<div class="alert alert-block alert-info">6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?</div>

---
<div class="alert alert-block alert-info">7. Which lessons are least accessed?</div>

---
<div class="alert alert-block alert-info">8. Others?</div>